In [1]:
For each department calculate the total items, maximum and minimum price of the subtotal , count the number of orders.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

In [3]:
import pyspark.sql.functions as f

In [4]:
o_items= spark.read.format('csv').options(header='True',inferSchema='true').load('/FileStore/tables/order_items.csv')
o_items=o_items.select('_c0')
o_items.show(5,False)

In [5]:
o=o_items.select(
f.split(o_items._c0,',')[0].cast('int').alias('o_item_id'),
f.split(o_items._c0,',')[1].cast('int').alias('o_item_order_id'),
f.split(o_items._c0,',')[2].cast('int').alias('o_item_product_id'),
f.split(o_items._c0,',')[3].cast('int').alias('quantity'),
f.split(o_items._c0,',')[4].cast('float').alias('subtotal'),
f.split(o_items._c0,',')[5].cast('float').alias('product_price')
  )

In [6]:
o.show(5,False)

In [7]:
products= spark.read.format('csv').options(header='False',inferSchema='true').load('/FileStore/tables/products.csv')

In [8]:
p=products.select(
f.split(products._c0,',')[0].cast('int').alias('product_id'),
f.split(products._c0,',')[1].cast('int').alias('product_cat_id'),
f.split(products._c0,',')[2].cast('string').alias('product_name'),
f.split(products._c0,',')[3].cast('string').alias('description'),
f.split(products._c0,',')[4].cast('float').alias('price'),
f.split(products._c0,',')[5].cast('string').alias('image')
)

In [9]:
p.show(3,False)

In [10]:
categories= spark.read.format('csv').options(header='False',inferSchema='true').load('/FileStore/tables/categories.csv')

In [11]:
c=categories.select(
f.split(categories._c0,',')[0].cast('int').alias('category_id'),
f.split(categories._c0,',')[1].cast('int').alias('category_dept_id'),
f.split(categories._c0,',')[2].cast('string').alias('category_name')
)

In [12]:
c.show(4,False)

In [13]:
departments= spark.read.format('csv').options(header='False',inferSchema='true').load('/FileStore/tables/departments.csv')

In [14]:
d=departments.select(
f.split(departments._c0,',')[0].cast('int').alias('dept_id'),
f.split(departments._c0,',')[1].cast('string').alias('dept_name'))

In [15]:
d.show(2,False)

In [16]:
o.join(p,o.o_item_product_id==p.product_id,"inner").\
join(c,c.category_id==p.product_cat_id,"inner").\
join(d,d.dept_id==c.category_dept_id,"inner").\
groupBy(d.dept_name).\
agg(f.sum(o.quantity).alias('Sum'),
f.max(o.subtotal).alias('Max'),
f.min(o.subtotal).alias('Min'),
f.count(o.o_item_id).alias('Count')).\
orderBy(d.dept_name).show()